In [ ]:
import numpy as np
import pandas as pd
import json
from google import genai

from google.oauth2.service_account import Credentials
from google.cloud import storage
import os


scopes = ["https://www.googleapis.com/auth/cloud-platform"]
SERVICE_ACCOUNT_FILE = "/Data2/Arun-UAV/NLP/new_cloud_coount.json"
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scopes)

client = storage.Client(credentials=credentials)

gen_client = genai.Client(
    vertexai=True,
    project='third-apex-476512-a7',   # or set directly
    location='us-central1',    # or set directly, e.g. "us-central1"
    credentials=credentials
)

In [147]:
batch_df = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/evidence_head_train_datasets/finecops_ref/train_processed_finecopes_ref_30k.csv")

In [148]:
batch_df

,image_id,answer,objects_info
0,2373626,"The dish, positioned to the right of the gray ...","[{'object_id': '3725798', 'names': ['dish'], '..."
1,285719,"On the left side of the metal and black fence,...","[{'object_id': '4401457', 'names': ['bike'], '..."
2,2383125,The post that is to the left of the car that i...,"[{'object_id': '534946', 'names': ['car'], 'h'..."
3,2349520,The clock that is to the left of the tower tha...,"[{'object_id': '3903072', 'names': ['roof'], '..."
4,2352548,The white keyboard positioned to the right of ...,"[{'object_id': '1807596', 'names': ['computer'..."
...,...,...,...
29995,2410239,"On the white glass, the white napkins are neat...","[{'object_id': '222683', 'names': ['bowl'], 'h..."
29996,2321990,The orange and old chair that is to the right ...,"[{'object_id': '3341883', 'names': ['bandana']..."
29997,2367927,A chair made of the same materials as the tabl...,"[{'object_id': '749365', 'names': ['chair'], '..."
29998,2355072,The yellow train that is near the green and ta...,"[{'object_id': '833081', 'names': ['building']..."


In [149]:
batch_df["answer"].iloc[0]

'The dish, positioned to the right of the gray bowl and further down from the red strawberry, rests elegantly on the table.'

In [150]:
batch_df["objects_info"].iloc[0]

"[{'object_id': '3725798', 'names': ['dish'], 'h': 131, 'w': 174, 'y': 1, 'x': 325}, {'object_id': '2428018', 'names': ['gray bowl'], 'h': 159, 'w': 179, 'y': 0, 'x': 0}, {'object_id': '2235351', 'names': ['red strawberry'], 'h': 50, 'w': 65, 'y': 73, 'x': 290}]"

In [ ]:
batch_df

In [12]:
final_df = batch_df[~batch_df["local_path"].isin(poc_df["local_path"].to_list())]

In [2]:
final_df = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_batch_1_15000_gcp_upload_urs.csv")
prompt_df = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/llava_gen_des_train.csv").sample(10000)

In [3]:
final_df.head(2)

,local_path,gcs_uri,error
0,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...,gs://train_data_vision_1/coco_batch_1_15000/CO...,NaN
1,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...,gs://train_data_vision_1/coco_batch_1_15000/CO...,NaN


In [4]:
prompt_df.head(2)

,image_id,answer,question,question_id,image_path
8803,COCO_train2014_000000559900.jpg,This indoor scene captures two young boys enjo...,Please describe this image in detail.,8803,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...
8660,COCO_train2014_000000463308.jpg,This outdoor scene captures a dynamic moment d...,Please describe this image in detail.,8660,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...


In [ ]:
prompt_df

# batch creation

In [5]:
def get_prompt(question, ans):
  prompt = f"""
You are given an **image** and its **caption** describing the visual scene.
Your job is to **analyze the caption** and extract all **meaningful, semantically important words** that can be visually grounded in the image.
Each extracted word must be categorized into one of the following classes:

* **objects** (car, tree, person, lamp)
* **attributes** (red, shiny, tall)
* **relations** (on, under, behind, next_to)
* **actions** (running, holding, sitting)
* **count** (two, many, three)
* **scene/context** (beach, kitchen, street)
* **decision tokens** (yes, no, true, false, present, absent, exist, not, visible, correct)

For every detected word, provide a structured JSON entry with its **category, word, related object IDs, and bounding box** coordinates.
If the word refers to a hallucinated (nonexistent) visual concept, set its bounding box as `[0, 0, 0, 0]`.

Note: always use image scale as 1000 * 1000 for bounding box coordinates.
---

### 🧾 **Expected Output Format**

```json
{{
  "objects": [
    {{"id": 1, "word": "dog", "bounding_box": [120, 150, 260, 310]}},
    {{"id": 2, "word": "ball", "bounding_box": [400, 230, 460, 280]}}
  ],
  "attributes": [
    {{"word": "brown", "objects_involved": [1]}},
    {{"word": "round", "objects_involved": [2]}}
  ],
  "relations": [
    {{"word": "holding", "objects_involved": [1, 2]}},
   {{"word": "on", "objects_involved": [1, 2]}}
  ],
  "count": [
    {{"word": "two", "objects_involved": [1, 2]}}
  ],
  "scene": [
    {{"word": "park", "bounding_box": [0, 0, 640, 480]}}
  ],
  "decision_tokens": [
   {{"word": "visible", "objects_involved": [1]}}
  ]
}}
```
Inputs:
<attached image>

Question: {question}

Caption: {ans}

---
"""
  return prompt

all_res = []
for inx, row in prompt_df.iterrows():
    question = row["question"]
    ans  = row["answer"]
    PROMPT = get_prompt(question, ans)
    img_path = row["image_path"]
    uri = final_df[final_df["local_path"] == img_path]["gcs_uri"].iloc[0]
    
    res = {"request":{"contents": [{"role": "user", "parts": [{"text": PROMPT}, {"fileData": {"fileUri": uri, "mimeType": "image/jpeg"}}]}], 
                      "generationConfig": {"temperature": 0, "topP": 1, "maxOutputTokens": 5000,"thinking_config":{"thinking_budget":1000}}}}
    all_res.append(res)

In [6]:
len(all_res)

10000

In [7]:

# PROMPT = """
# You are a specialist in rich and precise scene understanding.
# Given an input image, generate a comprehensive, contextually aware, and fluent description that captures all key visual elements, their relationships, emotions, and possible context or story.

# Your description should go beyond short captions — it must resemble a paragraph of visual storytelling that includes:

# Scene type: indoor/outdoor, environment, lighting, time of day
# Objects and entities: names, counts, shapes, colors, materials
# Actions and interactions: what the people or objects are doing
# Spatial layout: foreground, background, relative positions
# Emotions or atmosphere: tone, mood, aesthetics
# Possible context: what might be happening or implied by the scene

# Avoid generic or repetitive statements. Be vivid, factual, and coherent. Use natural language instead of bullet points.

# Output JSON format:
# {image_description: <full attached image description>}
# """

# all_res = []
# for uri in df["gcs_uri"].tolist():
#     res = {"request":{"contents": [{"role": "user", "parts": [{"text": PROMPT}, {"fileData": {"fileUri": uri, "mimeType": "image/jpeg"}}]}], 
#                       "generationConfig": {"temperature": 0.6, "topP": 1, "maxOutputTokens": 1000,"thinking_config":{"thinking_budget":0}}}}
#     all_res.append(res)

In [8]:
# def get_prompt(question, ans_1, ans_2):
#   prompt = f"""
# You are given an **image** and **two question–answer (QA) pairs** related to that image.
# Your job is to analyze each QA pair separately and return **two lists of single words (no phrases)**:

# 1. **`hallucination_candidates`** — all meaningful words that are semantically important for hallucination detection, belonging to categories like:

#    * **objects** (car, tree, person, lamp),
#    * **attributes** (red, shiny, tall),
#    * **relations** (on, under, behind, next_to),
#    * **actions** (running, holding, sitting),
#    * **count** (two, many, three),
#    * **scene/context** (beach, kitchen, street).
#    * **Decision tokens** yes, no, true, false, present, absent, exist, not, visible, correct

#    Include only **specific, content-bearing words** that could be visually verifiable or falsifiable from the image.
#    Do **not** include stopwords, determiners, or abstract terms.

# 2. **`hallucinated_words`** — a subset of the above list that are **not visually supported** or are **contradicted** by the image content (i.e., hallucinated terms).

# **Inputs:**

# <attached image>

# qa_pair_1:
# Question: {question}
# Answer: {ans_1}

# qa_pair_2:
# Question: {question}
# Answer: {ans_2}


# **Output format (strict JSON per QA pair):**

# ```json
# {{
#   "qa_pair_1": {{
#     "hallucination_candidates": ["word1", "word2", ...],
#     "hallucinated_words": ["wordX", "wordY", ...]
#   }},
#   "qa_pair_2": {{
#     "hallucination_candidates": ["word1", "word2", ...],
#     "hallucinated_words": ["wordX", "wordY", ...]
#   }}
# }}
# ```
# Note: extract "hallucination_candidates" and "hallucinated_words" from the Answer strictly not from the Question

# Ensure both lists contain **only lowercase single words** and are **deduplicated**.
# Focus on **important, visually grounded words** — not all tokens.

# ---
# """
#   return prompt

# all_res = []
# for inx, row in prompt_df.iterrows():
#     question = row["prompt"].replace("<image>", "")
#     nh_ans  = row["source_text"]
#     h_ans = row["hallucinated_text"]
#     PROMPT = get_prompt(question, nh_ans, h_ans)
#     img_path = f"/Data2/Arun-UAV/NLP/vision_halu/visual_genome/target_images/{row['image_id']}.jpg"
#     uri = df[df["local_path"] == img_path]["gcs_uri"].iloc[0]


#     res = {"request":{"contents": [{"role": "user", "parts": [{"text": PROMPT}, {"fileData": {"fileUri": uri, "mimeType": "image/jpeg"}}]}], 
#                       "generationConfig": {"temperature": 0.6, "topP": 1, "maxOutputTokens": 1000,"thinking_config":{"thinking_budget":0}}}}
#     all_res.append(res)

In [9]:
# def get_prompt(question, ans_1, ans_2):
#   prompt = f"""
# You are given an **image** and **two captions (descriptions)** related to that image.
# Your job is to analyze each caption separately and return **two lists of single words (no phrases)**:

# 1. **`hallucination_candidates`** — all meaningful words that are semantically important for hallucination detection, belonging to categories like:

#    * **objects** (car, tree, person, lamp),
#    * **attributes** (red, shiny, tall),
#    * **relations** (on, under, behind, next_to),
#    * **actions** (running, holding, sitting),
#    * **count** (two, many, three),
#    * **scene/context** (beach, kitchen, street).
#    * **Decision tokens** yes, no, true, false, present, absent, exist, not, visible, correct

#    Include only **specific, content-bearing words** that could be visually verifiable or falsifiable from the image.
#    Do **not** include stopwords, determiners, or abstract terms.

# 2. **`hallucinated_words`** — a subset of the above list that are **not visually supported** or are **contradicted** by the image content (i.e., hallucinated terms).

# **Inputs:**

# <attached image>

# caption_pair_1:
# Question: {question}
# Caption: {ans_1}

# caption_pair_2:
# Question: {question}
# Caption: {ans_2}


# **Output format (strict JSON per caption pair):**

# ```json
# {{
#   "caption_pair_1": {{
#     "hallucination_candidates": ["word1", "word2", ...],
#     "hallucinated_words": ["wordX", "wordY", ...]
#   }},
#   "caption_pair_2": {{
#     "hallucination_candidates": ["word1", "word2", ...],
#     "hallucinated_words": ["wordX", "wordY", ...]
#   }}
# }}
# ```
# Note: extract "hallucination_candidates" and "hallucinated_words" from the Answer strictly not from the Question

# Ensure both lists contain **only lowercase single words** and are **deduplicated**.
# Focus on **important, visually grounded words** — not all tokens.

# ---
# """
#   return prompt

# all_res = []
# for inx, row in prompt_df.iterrows():
#     question = row["prompt"].replace("<image>", "")
#     nh_ans  = row["source_text"]
#     h_ans = row["hallucinated_text"]
#     PROMPT = get_prompt(question, nh_ans, h_ans)
#     img_path = f"/Data2/Arun-UAV/NLP/vision_halu/visual_genome/target_images/{row['image_id']}.jpg"
#     uri = df[df["local_path"] == img_path]["gcs_uri"].iloc[0]

#     res = {"request":{"contents": [{"role": "user", "parts": [{"text": PROMPT}, {"fileData": {"fileUri": uri, "mimeType": "image/jpeg"}}]}], 
#                       "generationConfig": {"temperature": 0, "topP": 1, "maxOutputTokens": 2000,"thinking_config":{"thinking_budget":1000}}}}
#     all_res.append(res)

In [10]:
df = pd.DataFrame(all_res)
df.to_json("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/gemini_batch.jsonl", orient="records", lines=True)

# Uploading files to gcp

In [18]:
def upload_to_gcs(local_path: str, gcs_uri: str):
    """
    Upload a local file to a target GCS URI.

    Args:
        local_path (str): Local file path to upload.
        gcs_uri (str): Target GCS URI like 'gs://my-bucket/path/to/upload.txt'
        service_account_path (str): Path to GCP service account JSON.
    """
    if not gcs_uri.startswith("gs://"):
        raise ValueError("Invalid GCS URI. Must start with gs://")

    parts = gcs_uri[5:].split("/", 1)
    bucket_name = parts[0]
    blob_name = parts[1]

    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(local_path)

    print(f"✅ Uploaded {local_path} → {gcs_uri}")


In [21]:
upload_to_gcs(local_path="/Data2/Arun-UAV/NLP/vision_halu/train_datasets/gemini_batch.jsonl", gcs_uri = "gs://train_data_vision_1/gemini_batch_info/gemini_batch.jsonl")

✅ Uploaded /Data2/Arun-UAV/NLP/vision_halu/train_datasets/gemini_batch.jsonl → gs://train_data_vision_1/gemini_batch_info/gemini_batch.jsonl


In [22]:
# start batch processing

# start batch processing

In [23]:
import time

from google import genai
from google.genai.types import CreateBatchJobConfig, JobState, HttpOptions
output_uri = "gs://train_data_vision_1/gemini_batch_info/"

# See the documentation: https://googleapis.github.io/python-genai/genai.html#genai.batches.Batches.create
job = gen_client.batches.create(
    model="gemini-2.5-flash",
    src="gs://train_data_vision_1/gemini_batch_info/gemini_batch.jsonl",
    config=CreateBatchJobConfig(dest=output_uri),
)
print(f"Job name: {job.name}")
print(f"Job state: {job.state}")

Job name: projects/358874265041/locations/us-central1/batchPredictionJobs/8274965141032796160
Job state: JOB_STATE_PENDING


In [29]:
job_info = gen_client.batches.get(name=job.name)
job_info.state

<JobState.JOB_STATE_SUCCEEDED: 'JOB_STATE_SUCCEEDED'>

# download batch results

In [30]:
def download_from_gcs(gcs_uri: str, local_path: str):
    """
    Download a file from GCS based on its gs:// URI.

    Args:
        gcs_uri (str): GCS URI like 'gs://my-bucket/path/to/file.txt'
        local_path (str): Path to store the downloaded file locally.
    """
    # Parse bucket and blob name
    if not gcs_uri.startswith("gs://"):
        raise ValueError("Invalid GCS URI. Must start with gs://")

    parts = gcs_uri[5:].split("/", 1)
    bucket_name = parts[0]
    blob_name = parts[1]
    
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    blob.download_to_filename(local_path)

    print(f"✅ Downloaded {gcs_uri} → {local_path}")


In [31]:
download_from_gcs(gcs_uri="gs://train_data_vision_1/gemini_batch_info/prediction-model-2025-10-28T14:36:23.912008Z/predictions.jsonl", local_path ="/Data2/Arun-UAV/NLP/vision_halu/train_datasets/gemini_btach_res.jsonl")

✅ Downloaded gs://train_data_vision_1/gemini_batch_info/prediction-model-2025-10-28T14:36:23.912008Z/predictions.jsonl → /Data2/Arun-UAV/NLP/vision_halu/train_datasets/gemini_btach_res.jsonl


In [32]:
pred_data = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/gemini_btach_res.jsonl", lines=True)

In [100]:
def get_word_2_bbox(target_df, objects_df):
    bbox_map = {}
    for row in target_df.iterrows():
        word = row[1]["word"]
        obj_ids = row[1]["objects_involved"]
        bbox = objects_df[objects_df["id"].isin(obj_ids)]["bounding_box"].to_list()
        bbox_map[word] = bbox
    return bbox_map

In [116]:
from tqdm import tqdm

all_res = []
error = 0
for inx, row in tqdm(pred_data.iterrows()):
    try:
        img_name = row["request"]["contents"][0]["parts"][1]["fileData"]["fileUri"].split("/")[-1]
        res = eval(row["response"]["candidates"][0]["content"]["parts"][0]["text"].replace("json", "").strip("```"))
        objects_df = pd.DataFrame(res["objects"])
        
        target_words = {i:j for i, j in zip(objects_df["word"], objects_df["bounding_box"])}
        
        try:
            attr_df = pd.DataFrame(res["attributes"])
            if attr_df.shape[0]>0:
                attr_df = attr_df[attr_df["objects_involved"].apply(lambda x: len(x)!=0)]
                att_words = get_word_2_bbox(attr_df, objects_df)
                target_words.update(att_words)

        except Exception as e:
            error += 1
            
        try:
            rel_df = pd.DataFrame(res["relations"])
            if rel_df.shape[0]>0:
                rel_df = rel_df[rel_df["objects_involved"].apply(lambda x: len(x)!=0)]
                rel_words = get_word_2_bbox(rel_df, objects_df)
                target_words.update(rel_words)
        except Exception as e:
            error += 1
        
        try:
            count_df = pd.DataFrame(res["count"])
            if count_df.shape[0]>0:
                count_df = count_df[count_df["objects_involved"].apply(lambda x: len(x)!=0)]
                count_words = get_word_2_bbox(count_df, objects_df)
                target_words.update(count_words)
        except Exception as e:
            error += 1
        
        try:
            scene_df = pd.DataFrame(res["scene"])
            if scene_df.shape[0]>0:
                scene_words = {i:j for i, j in zip(scene_df["word"], scene_df["bounding_box"])}
                target_words.update(scene_words)
        except Exception as e:
            error += 1

        try:
            decision_df = pd.DataFrame(res["decision_tokens"])
            if decision_df.shape[0]>0:
                decision_df = decision_df[decision_df["objects_involved"].apply(lambda x: len(x)!=0)]
                decision_words = get_word_2_bbox(decision_df, objects_df)
            target_words.update(decision_words)
        except Exception as e:
            error += 1
        
        all_res.append({"image_name": img_name, "word_2_bbox": target_words})
    
    except Exception as e:
        error += 1

10000it [01:58, 84.43it/s]


In [127]:
clean_res_df = pd.DataFrame(all_res)
columns=["image_id", "word_2_bbox"]
clean_res_df.columns = columns

In [ ]:
final_res_df

In [129]:
final_res_df = pd.merge(clean_res_df, prompt_df, on = "image_id", how='left')

In [128]:
clean_res_df.head(2)

,image_id,word_2_bbox
0,COCO_train2014_000000507721.jpg,"{'grass': [0, 400, 1000, 1000], 'structure': [..."
1,COCO_train2014_000000560217.jpg,"{'cat': [150, 180, 850, 800], 'film stock labe..."


In [122]:
prompt_df.head(2)

,image_id,answer,question,question_id,image_path
8803,COCO_train2014_000000559900.jpg,This indoor scene captures two young boys enjo...,Please describe this image in detail.,8803,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...
8660,COCO_train2014_000000463308.jpg,This outdoor scene captures a dynamic moment d...,Please describe this image in detail.,8660,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...


In [136]:
final_res_df.head(2)

,image_id,word_2_bbox,answer,question,question_id,image_path
0,COCO_train2014_000000507721.jpg,"{'grass': [0, 400, 1000, 1000], 'structure': [...","This outdoor scene captures a rustic, rural la...",Please describe this image in detail.,5616,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...
1,COCO_train2014_000000560217.jpg,"{'cat': [150, 180, 850, 800], 'film stock labe...","This outdoor, dimly lit scene, captured likely...",Please describe this image in detail.,1966,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...


In [145]:
final_res_df = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/evidence_head_train_datasets/coco_long_captions/coco_img_des_10k_bb_annot.csv")

In [146]:
final_res_df.head(2)

,image_id,word_2_bbox,answer,question,question_id,image_path
0,COCO_train2014_000000507721.jpg,"{'grass': [0, 400, 1000, 1000], 'structure': [...","This outdoor scene captures a rustic, rural la...",Please describe this image in detail.,5616,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...
1,COCO_train2014_000000560217.jpg,"{'cat': [150, 180, 850, 800], 'film stock labe...","This outdoor, dimly lit scene, captured likely...",Please describe this image in detail.,1966,/Data2/Arun-UAV/NLP/vision_halu/train_datasets...


In [47]:
import re
import json

def extract_qa_pairs(text: str):
    """
    Extracts qa_pair_1 and qa_pair_2 (Question and Answer) from multiple prompts
    having the same structure.

    Args:
        text (str): Input text containing multiple prompt blocks.

    Returns:
        list[dict]: List of dictionaries, each containing qa_pair_1 and qa_pair_2 with Q/A.
    """
    # Pattern to match both caption_pair_1 and caption_pair_2 blocks
    pattern = re.compile(
        r'caption_pair_1:\s*Question:\s*(?P<q1>.*?)\s*Caption:\s*(?P<a1>.*?)\s*'
        r'caption_pair_2:\s*Question:\s*(?P<q2>.*?)\s*Caption:\s*(?P<a2>.*?)(?=\n\s*\n|$)',
        re.DOTALL | re.IGNORECASE
    )

    results = []
    for match in pattern.finditer(text):
        qa_dict = {
            "caption_pair_1": {
                "Question": match.group("q1").strip(),
                "Caption": match.group("a1").strip()
            },
            "caption_pair_2": {
                "Question": match.group("q2").strip(),
                "Caption": match.group("a2").strip()
            }
        }
        results.append(qa_dict)

    return results

In [109]:
all_res = []
error = 0
for inx, row in pred_data.iterrows():
    try:
        img_name = row["request"]["contents"][0]["parts"][1]["fileData"]["fileUri"].split("/")[-1]
        qa = extract_qa_pairs(row["request"]["contents"][0]["parts"][0]["text"].split("<attached image>")[-1].split("**Output format")[0])

        question = qa[0]["caption_pair_1"]['Question']
        answer_1 = qa[0]["caption_pair_1"]['Caption']
        answer_2 = qa[0]["caption_pair_2"]['Caption']

        img_dec = row["response"]["candidates"][0]["content"]["parts"][0]["text"]
        c_img_dec = eval(img_dec.replace("json", "").strip("```"))

        all_res.append([img_name, question, answer_1,  c_img_dec["caption_pair_1"]["hallucination_candidates"], c_img_dec["caption_pair_1"]["hallucinated_words"]])
        all_res.append([img_name, question, answer_2,  c_img_dec["caption_pair_2"]["hallucination_candidates"], c_img_dec["caption_pair_2"]["hallucinated_words"]])
    except Exception as e:
        error += 1

In [110]:
result_df = pd.DataFrame(all_res)

In [112]:
result_df.shape

(27986, 5)

In [111]:
result_df.columns = ["image_path", "question", "answer", "candidates", "hallucination_candidates"]

In [113]:
result_df.to_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/caption/gemini_labeled_28k.csv", index=False)

In [114]:
result_df

,image_path,question,answer,candidates,hallucination_candidates
0,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"[one, woman, holding, bat, playing, tennis, ba...",[bat]
1,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"[one, woman, holding, bat, playing, tennis, fr...","[bat, frisbee]"
2,2378712.jpg,What do you think is going on in this snapshot?,In this picture we can see a bus on road and a...,"[bus, road, footpath, statue, person, building...",[]
3,2378712.jpg,What do you think is going on in this snapshot?,In this picture we can see a bus on road and a...,"[bus, road, footpath, statue, person, building...",[trains]
4,2379616.jpg,Describe the following image.,There is a person standing on a bicycle and th...,"[person, standing, bicycle, car, beside, sitti...","[standing, sitting, bike, left, corner, few, r..."
...,...,...,...,...,...
27981,2398155.jpg,What do you see happening in this image?,"An older woman in a black, white, and gray shi...","[older, woman, black, white, gray, shirt, pant...","[gray, apron, five, stove]"
27982,2347016.jpg,Write a detailed description of the given image.,In this picture we can see a black color cat s...,"[black, cat, sitting, laptop, on, book, pencil...","[cd, remote]"
27983,2347016.jpg,Write a detailed description of the given image.,In this picture we can see a black color cat s...,"[black, cat, sitting, laptop, on, book, pencil...","[left, cd, remote, cup]"
27984,2316274.jpg,Can you elaborate on the elements of the pictu...,In this image there is a dog standing in a roo...,"[dog, standing, room, carpet, books, tables, c...","[standing, toys]"


In [ ]:
all_res = []
for inx, row in pred_data.iterrows():
    try:
        img_name = row["request"]["contents"][0]["parts"][1]["fileData"]["fileUri"].split("/")[-1]
        img_dec = row["response"]["candidates"][0]["content"]["parts"][0]["text"]
        c_img_dec = eval(img_dec.replace("json", "").strip("```"))["image_description"]
        all_res.append({"image": img_name, "description": c_img_dec})
    except Exception as e:
        print(f"Error processing row {inx}: {e}")

In [39]:
df = pd.DataFrame(all_res)

In [41]:
df.head(2)

,image,description
0,COCO_train2014_000000467172.jpg,The indoor scene captures two domestic cats ex...
1,COCO_train2014_000000328023.jpg,This outdoor scene captures a lively moment in...


In [42]:
old_poc_df  = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_5000_train_with_gemini_des.csv")

In [92]:
df.to_csv("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_5000_train_with_gemini_des.csv", index=False)

In [43]:
old_poc_df.head(2)

,image,description
0,COCO_train2014_000000078572.jpg,This outdoor scene captures a bustling street ...
1,COCO_train2014_000000401963.jpg,The image captures an outdoor scene under brig...


In [44]:
final_df_15k = pd.concat([old_poc_df, df])

In [46]:
final_df_15k["image"].nunique()

14579

In [47]:
final_df_15k.to_csv("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_batch_1_15000_train_with_gemini_des.csv", index=False)